In [3]:
import duckdb

In [5]:
con = duckdb.connect(database="dados_duckdb.db", read_only=False)

In [26]:
df = con.execute("""
                 select * from (
                 select *, row_number() over (partition by NATBR order by data_ingestao desc) as row
                  from 
                 bronze_z0019
                 where data_ingestao >= '2025-09-23'
                 ) where row = 1
                 """).fetchdf()

df.head(10)


,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao,row
0,10005,MACHADO,BT50,100,100,z0019_2.csv,2025-09-23 21:24:03.660288,1
1,10004,SERRA,BT50,100,200,z0019_2.csv,2025-09-23 21:24:03.660288,1
2,10002,MARTELO,BT50,100,1500,z0019_1.csv,2025-09-23 21:23:49.511440,1
3,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-09-23 21:23:49.511440,1
4,10003,PREGO,BT10,100,60,z0019_2.csv,2025-09-23 21:24:03.660288,1


In [27]:
df_final = df.drop(columns=['nome_arquivo', 'data_ingestao', 'row'])



In [28]:
df_final.head(5)

,NATBR,MAKTX,WERKS,MAINS,LABST
0,10005,MACHADO,BT50,100,100
1,10004,SERRA,BT50,100,200
2,10002,MARTELO,BT50,100,1500
3,10001,PARAFUSO,BT10,100,100
4,10003,PREGO,BT10,100,60


In [29]:
df_final = df_final.rename(columns={'NATBR': 'id'})
df_final = df_final.rename(columns={'MAKTX': 'nm_produto'})
df_final = df_final.rename(columns={'WERKS': 'id_categoria'})
df_final = df_final.rename(columns={'MAINS': 'id_fornecedor'})
df_final = df_final.rename(columns={'LABST': 'vl_preco'})
df_final.head(5)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10005,MACHADO,BT50,100,100
1,10004,SERRA,BT50,100,200
2,10002,MARTELO,BT50,100,1500
3,10001,PARAFUSO,BT10,100,100
4,10003,PREGO,BT10,100,60


In [33]:
df2 = df_final

df2 = df2.astype(
    {
    'id': int,
    'nm_produto': str,
    'id_categoria': str,
    'id_fornecedor' : int,
    'vl_preco': float
    }
)

df2.dtypes

id                 int64
nm_produto        object
id_categoria      object
id_fornecedor      int64
vl_preco         float64
dtype: object

In [34]:
con.execute("""
CREATE TABLE IF NOT EXISTS produtos(
            id BIGINT,
            nm_produto TEXT,
            id_categoria TEXT,
            id_fornecedor BIGINT,
            vl_preco FLOAT
            )
""")

In [35]:
df2.head()

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10005,MACHADO,BT50,100,100.0
1,10004,SERRA,BT50,100,200.0
2,10002,MARTELO,BT50,100,1500.0
3,10001,PARAFUSO,BT10,100,100.0
4,10003,PREGO,BT10,100,60.0


In [37]:
con.execute('INSERT INTO produtos SELECT * FROM df2')

In [38]:
df_resultado = con.execute("select * from produtos").fetchdf()
df_resultado.head()

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10005,MACHADO,BT50,100,100.0
1,10004,SERRA,BT50,100,200.0
2,10002,MARTELO,BT50,100,1500.0
3,10001,PARAFUSO,BT10,100,100.0
4,10003,PREGO,BT10,100,60.0


In [39]:
con.close()